# Create CIHR Awards

Creates CIHR (Canadian Institutes of Health Research) awards from open data. ~82K unique awards.

**Prerequisites:**
- Run `scripts/local/cihr_to_s3.py` to download and upload the data first.

**Data source:** https://open.canada.ca/data/en/dataset/49edb1d7-5cb4-4fa7-897c-515d1aad5da3  
**S3 location:** `s3a://openalex-ingest/awards/cihr/cihr_projects.parquet`

**CIHR funder:**
- funder_id: 4320334506
- display_name: "Canadian Institutes of Health Research"
- ROR: https://ror.org/01gavpb45
- DOI: 10.13039/501100000024

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.cihr_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/cihr/cihr_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~82K)
SELECT COUNT(*) as total_projects FROM openalex.awards.cihr_raw;

In [ ]:
%sql
-- Inspect the schema
DESCRIBE openalex.awards.cihr_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.cihr_raw LIMIT 5;

## Step 2: Create CIHR Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.cihr_awards
USING delta
AS
WITH
-- Get CIHR funder from OpenAlex by explicit funder_id
cihr_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320334506  -- Canadian Institutes of Health Research
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:funding_reference_number
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(TRIM(g.funding_reference_number))))) % 9000000000 as id,

        -- Display name = application title
        g.application_title as display_name,

        -- Description = abstract
        g.application_abstract as description,

        -- Funder info
        f.funder_id,
        TRIM(g.funding_reference_number) as funder_award_id,

        -- Amount
        TRY_CAST(g.total_amount_awarded AS DOUBLE) as amount,
        'CAD' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map CIHR program types to OpenAlex types
        CASE
            WHEN LOWER(g.program_type) LIKE '%award%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%doctoral%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%postdoctoral%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%fellowship%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%scholarship%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%master%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%vanier%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%banting%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%canada research chair%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%salary%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%new investigator%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%career%' THEN 'fellowship'
            WHEN LOWER(g.program_name) LIKE '%travel%' THEN 'travel'
            WHEN LOWER(g.program_name) LIKE '%meeting%' THEN 'travel'
            WHEN LOWER(g.program_name) LIKE '%planning%' THEN 'planning'
            WHEN LOWER(g.program_name) LIKE '%operating grant%' THEN 'research'
            WHEN LOWER(g.program_name) LIKE '%project grant%' THEN 'research'
            WHEN LOWER(g.program_name) LIKE '%team grant%' THEN 'research'
            WHEN LOWER(g.program_name) LIKE '%foundation grant%' THEN 'research'
            WHEN LOWER(g.program_name) LIKE '%bridge%' THEN 'research'
            WHEN LOWER(g.program_name) LIKE '%catalyst%' THEN 'research'
            WHEN LOWER(g.program_name) LIKE '%randomized controlled%' THEN 'research'
            WHEN LOWER(g.program_type) LIKE '%grant%' THEN 'grant'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = program name
        g.program_name as funder_scheme,

        -- Provenance
        'cihr_opendata' as provenance,

        -- Dates
        TRY_TO_DATE(g.funding_start_date) as start_date,
        TRY_TO_DATE(g.funding_end_date) as end_date,
        YEAR(TRY_TO_DATE(g.funding_start_date)) as start_year,
        YEAR(TRY_TO_DATE(g.funding_end_date)) as end_year,

        -- Lead investigator - CIHR has structured name fields
        CASE
            WHEN g.family_name IS NOT NULL AND TRIM(g.family_name) != '' THEN
                struct(
                    TRIM(g.first_name) as given_name,
                    TRIM(g.family_name) as family_name,
                    CAST(NULL AS STRING) as orcid,
                    TRY_TO_DATE(g.funding_start_date) as role_start,
                    struct(
                        TRIM(g.research_institution_name) as name,
                        'Canada' as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available in CIHR open data at this level)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL - CIHR open data portal
        'https://open.canada.ca/data/en/dataset/49edb1d7-5cb4-4fa7-897c-515d1aad5da3' as landing_page_url,

        -- No DOI for CIHR grants
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(TRIM(g.funding_reference_number))))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.cihr_raw g
    CROSS JOIN cihr_funder f
    WHERE g.funding_reference_number IS NOT NULL
      AND TRIM(g.funding_reference_number) != ''
)

SELECT * FROM awards_transformed;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~82K)
SELECT COUNT(*) as total_cihr_awards FROM openalex.awards.cihr_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    start_date,
    lead_investigator
FROM openalex.awards.cihr_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be CIHR)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.cihr_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.cihr_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (top 20 funding programmes)
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.cihr_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_abstract,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(lead_investigator) as has_pi,
    ROUND(COUNT(display_name) * 100.0 / COUNT(*), 1) as pct_title,
    ROUND(COUNT(description) * 100.0 / COUNT(*), 1) as pct_abstract,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_amount,
    ROUND(COUNT(start_date) * 100.0 / COUNT(*), 1) as pct_dates
FROM openalex.awards.cihr_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt
FROM openalex.awards.cihr_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 25;

In [ ]:
%sql
-- Check lead institutions (top 20)
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as cnt
FROM openalex.awards.cihr_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check total funding amount
SELECT 
    ROUND(SUM(amount) / 1000000000, 2) as total_funding_billions_cad,
    ROUND(AVG(amount), 0) as avg_award_cad,
    ROUND(PERCENTILE_APPROX(amount, 0.5), 0) as median_award_cad
FROM openalex.awards.cihr_awards
WHERE amount IS NOT NULL;